In [11]:
import os


In [12]:
%pwd

'c:\\Users\\dell\\Documents\\mlops\\walmart-sales'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\dell\\Documents\\mlops\\walmart-sales'

In [4]:
import random
random.randint(1, 96)

60

In [13]:
from dataclasses import dataclass, field
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    pipeline_name: str
    model_instance_name: str
    # alpha: float
    # l1_ratio: float
    target_column: str

In [14]:
from projectFiles.constants import *
from projectFiles.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema
        # params = self.params

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            pipeline_name = config.pipeline_name,
            model_instance_name = config.model_instance_name,
            # alpha = params.alpha,
            # l1_ratio = params.l1_ratio,
            target_column = schema.TARGET_COLUMN.name
        )

        return model_trainer_config

In [16]:
import numpy as np
import pandas as pd
import pickle
from lightgbm import LGBMRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from projectFiles import logger

In [17]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        target_col = self.config.target_column

        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)

        train_df.drop(columns=["Date"], inplace=True)
        test_df.drop(columns=["Date"], inplace = True)

        categorical_cols = [c for c in train_df.columns if train_df[c].dtype in [object]]
        numerical_cols = [c for c in train_df.columns if train_df[c].dtype in [float, int] and c != target_col]
        cycl_num_cols = [c for c in train_df.columns if ("sin" in str(c)) or ("cos" in str(c))]

        train_x = train_df.drop(columns=target_col)
        train_y = train_df[target_col]
        test_x = test_df.drop(columns=target_col)
        test_y = test_df[target_col]

        pipeline = make_pipeline(ColumnTransformer([("num", MinMaxScaler(), [c for c in numerical_cols if c not in cycl_num_cols])]),
                                 LGBMRegressor(n_jobs=-1, random_state=100))
        regressor = pipeline.fit(train_x, train_y)
        model_instance = regressor.named_steps["lgbmregressor"]

        pickle.dump(regressor, open(os.path.join(self.config.root_dir, self.config.pipeline_name), 'wb'))
        logger.info(f"Training Pipeline successfully saved at {self.config.root_dir}/{self.config.pipeline_name}")
        pickle.dump(model_instance, open(os.path.join(self.config.root_dir, self.config.model_instance_name), 'wb'))
        logger.info(f"Model Instance successfully saved at {self.config.root_dir}/{self.config.model_instance_name}")

In [18]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

DEBUG: YAML Content from config\config.yaml -> {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/rkdsai/walmart-sales/raw/refs/heads/main/raw_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'data_dirs': {'features': 'artifacts/data_ingestion/features.csv', 'stores': 'artifacts/data_ingestion/stores.csv', 'train': 'artifacts/data_ingestion/train.csv', 'test': 'artifacts/data_ingestion/test.csv'}, 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_dirs': {'features': 'artifacts/data_ingestion/features.csv', 'stores': 'artifacts/data_ingestion/stores.csv', 'train': 'artifacts/data_ingestion/train.csv', 'test': 'artifacts/data_ingestion/test.csv'}}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_